In [ ]:
from datetime import date, datetime
from elasticsearch import Elasticsearch
import subprocess
import time

cmd = "ps -eo pid,%mem,%cpu"
name1 = "operating"
name2="operating-system"
es = Elasticsearch()

def get_doc(pid, mem, cpu, now):
    return { 
        "timestamp": now,
        "cpu": float(cpu),
        "mem": float(mem),
        "pid": pid
    }

def lines_of_process(line):
    sp = line.split() # Splits line into columns
    now = datetime.now()
    return get_doc(sp[0], sp[1], sp[2], now)

while True:
    out = subprocess.getoutput(cmd) 
    lines = out.splitlines() 
    cpu = 0
    mem = 0
    for x in map(lines_of_process, out.splitlines()[1:]):
        es.index(index=name1, body=x) # Pushing data to elastisearch
        cpu += x["cpu"]
        mem += x["mem"]
    es.index(index=name2, body={"mem": mem,
                                    "cpu": cpu, "timestamp": datetime.now()})

    time.sleep(1) 